In [1]:
import numpy as np
import scipy.spatial.distance as sd

from distance import SquaredL2
from neighborhood import neighbor_graph, laplacian
from correspondence import Correspondence
from stiefel import *

import torch
import torch.nn as nn
import torch.nn.functional as F
torch.set_default_tensor_type('torch.cuda.FloatTensor')

from datareader import *

import os.path

import pdb
cuda = torch.device('cuda') 

In [2]:
"""Defines the neural network"""

class Net(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(Net, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H1)
        self.linear2 = torch.nn.Linear(H1, H1)
        self.linear3 = torch.nn.Linear(H1, H1)
        self.linear4 = torch.nn.Linear(H1, D_out)
        self.relu = nn.LeakyReLU(0.01)
        
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        y_pred = self.linear4(x)
        return y_pred

In [3]:
"""Compute the projected gradient onto the tangent space of Stiefel Manifold"""

def skew(M,Z):
    ''' return the skew-symmetric part of $M^T Z$'''
    return 0.5 * (M.t()@Z - Z.t()@M)

def proj_stiefel(M,Z):
    ''' M is a d-by-r point on the stiefel manifold, defining a tangent
    space $T_M \mathcal{O}^{d \times r}$
    $Z \in \mathbb{R}^{d\times r}$ is an arbitrary point 
    we would like to project onto the '''
    MskewMTZ = M@skew(M,Z)
    IMMTZ = (torch.eye(len(M)) - M@M.t())@Z
    return MskewMTZ + IMMTZ

In [4]:
# Load patient data from excel file if there isn't any
data_file = './data/DER-01_PEC_Gene_expression_matrix_normalized.txt'
metadata_file = './data/PEC_Capstone_clinical_meta - PEC_Capstone_clinical_meta.csv'
if (not os.path.isfile('./data/gene_data.npy')):
    
    x, y, patient_id, feature_name = read_data(data_file, metadata_file)
    
    np.save('./data/gene_data.npy', x)
    np.save('./data/label.npy', y)
    np.save('./data/patient_id.npy', patient_id)
    np.save('./data/feature_name.npy', feature_name)
else:
    x = np.load('./data/gene_data.npy')
    y = np.load('./data/label.npy')
    patient_id = np.load('./data/patient_id.npy')
    feature_name = np.load('./data/feature_name.npy')

print('Shape of data: ', x.shape)

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.

x1_np = x[y == 0, :].T
x2_np = x[y == 1, :].T

# Take out a small bunch for debugging purpose
x1_np = x1_np[0:10000, :]
x2_np = x2_np[0:10000, :]

N, H1, H2, D_out = x1_np.shape[0], 1024, 512, 10

D_in1 = x1_np.shape[1]
D_in2 = x2_np.shape[1]

print(x1_np.shape)
print(x2_np.shape)

Shape of data:  (1814, 43885)
(10000, 1556)
(10000, 258)


In [5]:
def my_neighbor(x, bsize, k):
    """my_neighbor
    :param x: n x d numpy array
    """
    
    #x = torch.from_numpy(x).cuda()
    n = x.shape[0]
    n_ite = int(np.ceil(n*1.0 / bsize))
    
    adj_matrix = np.zeros((n, n), dtype=np.int32)
    
    for i in range(n_ite):
        if (i % 100 == 0):
            print(i)
        
        batch_range = range(i*bsize, (i+1)*bsize)
        if (batch_range[-1] > n):
            batch_range = range(i*bsize, n)
        
        x_i = x[batch_range, :]
        
        d_i = sd.cdist(x_i, x)
        
        k_nearest_i = np.argsort(d_i, axis=1)[:, 1:(k+1)]
        #k_nearest_matrix[batch_range, :] = k_nearest_i
        for j in batch_range:
            #pdb.set_trace()
            #ind = np.array(list(zip(batch_range, k_nearest_i[j,:])))
            try:
                adj_matrix[j, k_nearest_i[j%bsize,:]] = 1
                adj_matrix[k_nearest_i[j%bsize,:], j] = 1
            except Exception as e:
                pdb.set_trace()
    
    return adj_matrix

In [6]:
if (not os.path.isfile('./data/adj1_%d.npy' % N)):
    print('Computing neighbor graph...')
    #adj1 = neighbor_graph(x1_np, k=5)
    #adj2 = neighbor_graph(x2_np, k=5)
    
    adj1 = my_neighbor(x1_np, bsize=10, k=5)
    adj2 = my_neighbor(x2_np, bsize=10, k=5)
    
    print('Finished computing neighbor graph')
    np.save('./data/adj1_%d.npy' % N, adj1)
    np.save('./data/adj2_%d.npy' % N, adj2)
else:
    print('Load computed neighbor graph...')
    adj1 = np.load('./data/adj1_%d.npy' % N)
    adj2 = np.load('./data/adj2_%d.npy' % N)

corr = Correspondence(matrix=np.eye(N))

w = np.block([[corr.matrix(),adj1],
              [adj2, corr.matrix()]])

print('Computing Laplacian...')
L_np = laplacian(w, normed=True)
L = torch.from_numpy(L_np.astype(np.float32)).cuda()
print('Finished computing Laplacian')

Load computed neighbor graph...
Computing Laplacian...
Finished computing Laplacian


In [7]:
torch.manual_seed(2019)
np.random.seed(2019)

# Construct our model by instantiating the class defined above.
model1 = Net(D_in1, H1, D_out)
model2 = Net(D_in2, H1, D_out)

model1.cuda()
model2.cuda()

# Construct an Optimizer
params = list(model1.parameters()) + list(model2.parameters())
#optimizer = torch.optim.SGD(params, lr = 0.004)
optimizer = torch.optim.Adam(params, lr = 0.001)

random_stiefel = rand_stiefel(N*2, D_out)

# Putting x1 and x2 to tensor
x1 = torch.from_numpy(x1_np).cuda()
x2 = torch.from_numpy(x2_np).cuda()

batch_size = 256

n_ite =int(np.ceil(N*2.0 / batch_size));
n_epoch = 200

for e in range(n_epoch):
    for t in range(n_ite):
        batch_range = range(t*batch_size, (t+1)*batch_size)
        if (batch_range[-1] > N*2):
            batch_range = range(t*batch_size, N*2)

        # Forward pass: Compute predicted y by passing x to the model
        optimizer.zero_grad()

        y1_pred = model1(x1)
        y2_pred = model2(x2)

        outputs = torch.cat((y1_pred, y2_pred), 0)
        outputs = outputs[batch_range]

        loss = torch.norm(outputs - random_stiefel[batch_range, :])

        loss.backward()
        optimizer.step()
        
    if (e % 10 == 0):
        print('[%04d] Loss: %.5f' % (e+1, loss.cpu().data))

[0001] Loss: 0.13100
[0011] Loss: 0.10538
[0021] Loss: 0.09376
[0031] Loss: 0.11597
[0041] Loss: 0.12343
[0051] Loss: 0.09593
[0061] Loss: 0.09286
[0071] Loss: 0.06392
[0081] Loss: 0.06423
[0091] Loss: 0.06438
[0101] Loss: 0.05567
[0111] Loss: 0.09116
[0121] Loss: 0.06549
[0131] Loss: 0.05904
[0141] Loss: 0.05573
[0151] Loss: 0.07230
[0161] Loss: 0.08181
[0171] Loss: 0.06186
[0181] Loss: 0.08137
[0191] Loss: 0.07379


In [17]:
torch.manual_seed(2019)
np.random.seed(2019)

# Construct an Optimizer
params = list(model1.parameters()) + list(model2.parameters())
optimizer = torch.optim.SGD(params, lr = 0.02)
#optimizer = torch.optim.Adam(params, lr = 0.001)
batch_size = 128
n_epoch = 500
n_ite = int(np.ceil(n_epoch * 1.0 / batch_size))

for e in range(500):
    accum_grad = torch.zeros((256, 10)).cuda()
    for t in range(n_ite):
        batch_range = range(t*batch_size, (t+1)*batch_size)
        if (batch_range[-1] > N):
            batch_range = range(t*batch_size, N)
        L_batch_range = np.concatenate((batch_range, np.asarray(batch_range) + N), 0)
        # Forward pass: Compute predicted y by passing x to the model
        try:
            y1_pred = model1(x1[batch_range, :])
            y2_pred = model2(x2[batch_range, :])
        except Exception as e:
            pdb.set_trace()

        outputs = torch.cat((y1_pred, y2_pred), 0)

        # Project the output onto Stiefel Manifold
        u, s, v = torch.svd(outputs, some=True)
        proj_outputs = u@v.t()

        # Compute and print loss
        loss = torch.trace(proj_outputs.t()@L[L_batch_range,:][:, L_batch_range]@proj_outputs)
0
        # Zero gradients, perform a backward pass, and update the weights.
        proj_outputs.retain_grad()

        optimizer.zero_grad()
        loss.backward(retain_graph=True)

        # Project the (Euclidean) gradient onto the tangent space of Stiefel Manifold (to get Rimannian gradient)
        rgrad = proj_stiefel(proj_outputs, proj_outputs.grad) 
        accum_grad += rgrad
        optimizer.zero_grad()
        
    # Backpropogate the Rimannian gradient w.r.t proj_outputs
    proj_outputs.backward(accum_grad)
    accum_grad[:] = 0
    
    optimizer.step()

    if (e % 10 == 9):
        print(e+1, loss.item())

10 9.628013610839844
20 9.608041763305664
30 9.6057767868042
40 9.624540328979492
50 9.59405517578125
60 9.599438667297363
70 9.60513687133789
80 9.612357139587402
90 9.602792739868164
100 9.609492301940918
110 9.61546516418457
120 9.610350608825684
130 9.600461959838867
140 9.603946685791016
150 9.587674140930176
160 9.606496810913086
170 9.59586238861084
180 9.581197738647461
190 9.601890563964844
200 9.576210021972656
210 9.609225273132324
220 9.573551177978516
230 9.614559173583984
240 9.59571361541748
250 9.585641860961914
260 9.57463550567627
270 9.593141555786133
280 9.591930389404297
290 9.59908676147461
300 9.581263542175293
310 9.601405143737793
320 9.584795951843262
330 9.587654113769531
340 9.568805694580078
350 9.57101821899414
360 9.591824531555176
370 9.587491035461426
380 9.570846557617188
390 9.578156471252441
400 9.570921897888184
410 9.583796501159668
420 9.552523612976074
430 9.575945854187012
440 9.55333423614502
450 9.570051193237305
460 9.558344841003418
470 9.57